### Imports y dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from causallearn.utils.Dataset import load_dataset
from causallearn.search.ConstraintBased.PC import pc

from pgmpy.estimators import BicScore, K2Score, AICScore
from pgmpy.estimators import HillClimbSearch

from generateSyntheticDataset import generateRandomDagAndData

numberOfNodes = 5
edgeProb = 0.5
numberOfDatasets = 3
numberOfSamplesPerDataset = 200

In [2]:
data1 = pd.read_csv("/home/joaquin/Documents/Asignaturas/TFG UCO/Introduction-to-Causal-Inference/datasets/heart-disease(320k_rows)/2022/heart_2022_no_nans.csv")
data2 = pd.read_csv("/home/joaquin/Documents/Asignaturas/TFG UCO/Introduction-to-Causal-Inference/datasets/Agriculture Crop Yield(1M_rows)/crop_yield.csv")

datasets = {'Heart disease': data1, 'Agriculture Crop Yield': data2}

In [3]:
for dataset in datasets:
    # Drop non categorical columns
    dataset.drop(dataset.dtypes[dataset.dtypes.apply(lambda x: x not in (bool, object))].index, axis=1, inplace=True)
    
# Stay with just a sample of each dataset
datasets = {key: df.sample(n=numberOfSamplesPerDataset, random_state=0) for key, df in datasets.items()}

## Algorithms

### Hill Climbing

In [2]:
models_hc = {}

datasets = {'synthetic1': pd.read_csv('data/synthetic1.csv')}

for name, dataset in datasets.items():
    hc = HillClimbSearch(dataset)
    models_hc[name] = hc.estimate(scoring_method=BicScore(dataset), epsilon=0.01, max_iter=100)
    print(f"'{name}' dataset finished")
    print(models_hc[name].edges)

  0%|          | 0/100 [00:00<?, ?it/s]

'synthetic1' dataset finished
[('0', '4'), ('1', '6'), ('1', '8'), ('1', '5'), ('3', '5'), ('4', '7'), ('6', '5'), ('6', '3'), ('7', '9'), ('8', '6')]


### PC

In [9]:
from pgmpy.estimators import PC

datasets = {'synthetic1': pd.read_csv('data/synthetic1.csv')}

models_pc = {}
for name, dataset in datasets.items():
    pc = PC(dataset)
    graph = pc.estimate(significance_level=0.01).to_directed()
    models_pc[name] = graph
    print(graph.edges)

  0%|          | 0/5 [00:00<?, ?it/s]

[('6', '5'), ('6', '8'), ('1', '8'), ('1', '5'), ('3', '5'), ('4', '0'), ('7', '4'), ('9', '7')]


## Scores

In [10]:
def printScoreForModels(datasets, score):
    for name, dataset in datasets.items():
        print('********************************************')
        print(f"Dataset: {name}")
        print(f"{score.__name__} from HillClimbSearch: {score(dataset).score(models_hc[name])}")
        print(f"{score.__name__} from PC: {score(dataset).score(models_pc[name])}")
        # print(f"{score.__name__} from GES: {score(dataset).score(models_ges[name])}")
    
printScoreForModels(datasets, BicScore)
print('\n')
printScoreForModels(datasets, K2Score)
print('\n')
printScoreForModels(datasets, AICScore)

********************************************
Dataset: synthetic1
BicScore from HillClimbSearch: -58149.79766739368
BicScore from PC: -58164.08764887776


********************************************
Dataset: synthetic1
K2Score from HillClimbSearch: -58132.5908052942
K2Score from PC: -58149.14626473762


********************************************
Dataset: synthetic1
AICScore from HillClimbSearch: -58059.66841274398
AICScore from PC: -58081.16873460003
